In [3]:
# Eval 4input_40000 model
import numpy as np
from PIL import Image
import sys
sys.path.insert(0,'./caffe/python')
import caffe
import os

import scipy.io as sio    
import matplotlib.pyplot as plt  
#%matplotlib inline
plt.rcParams['figure.figsize']=[15,15]
import time
import scipy.misc

INPUT_CHANNEL = 4
MEAN = [114.578, 115.294, 108.353]
DATASET = 'douyu_7100'
n_cl = 2

# load net
DEPLOY = './experiment/exper_douyu2700_googlenet_batchsize_4input/deploy_ori.prototxt'
CAFFEMODEL = './experiment/exper_douyu2700_googlenet_batchsize_4input/snapshot/douyu_2700_train_iter_40000.caffemodel'
net = caffe.Net(DEPLOY, CAFFEMODEL, caffe.TEST)
caffe.set_mode_gpu();
caffe.set_device(0);

imglist = open('./data/'+DATASET+'/test5000.txt').readlines()
datanum = len(imglist)

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n,n)

hist = 0

for img in imglist:
    img = img.strip()
    
    # load image, switch to BGR, subtract mean, and make dims C x H x W for Caffe
    im = Image.open('./data/'+DATASET+'/Images/'+img+'.png')
    in_ = np.array(im, dtype=np.float32)
    in_ = in_[:,:,::-1]
    in_ -= np.array(MEAN)
    in_ = in_.transpose((2,0,1))
    
    # load label
    label = sio.loadmat('./data/'+DATASET+'/Labels/'+img+'.mat')['label']
    label[label!=0]==1
    
    if INPUT_CHANNEL==4:
        imshape = (368, 640, 4)
        in_batch = np.ndarray(shape=(1, imshape[2], imshape[1], imshape[0]), dtype=np.float32)
        in_batch[0,0:3,:,:] = in_.copy()
        
        # load 4-th channel
        im_pred = Image.open('./data/'+DATASET+'/pred_xijin/{}.png_pred.png'.format(img))
        in_ = np.array(im_pred, dtype=np.float32)
        in_ -= 128
        in_batch[0,3,:,:] = in_.copy()
        
    # shape for input (data blob is N x C x H x W), set data
    net.blobs['data'].reshape(*in_batch.shape)
    net.blobs['data'].data[...] = in_batch

    # run net and take argmax for prediction
    net.forward()
    out = net.blobs['score'].data[0].argmax(axis=0)
    
    this_hist =  fast_hist(label.flatten(),out.flatten(),n_cl)
    this_hist = np.array(this_hist, dtype=np.float32)
    this_iu = np.diag(this_hist) / (this_hist.sum(1) + this_hist.sum(0) - np.diag(this_hist))
    if np.isnan(this_iu[0]):
        this_iu[0] = 1
    if np.isnan(this_iu[1]):
        this_iu[1] = 1
    print img, this_iu, np.mean(this_iu) # first is neg iou, second is pos iou 
    hist += this_hist
    
hist = np.array(hist, dtype=np.float32)
iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
print '================================= result ================================='
print 'INPUT_CHANNEL: ',INPUT_CHANNEL
print 'MEAN', MEAN
print 'DEPLOY: ',DEPLOY
print 'CAFFEMODEL: ',CAFFEMODEL
print 'TEST DATASET: ',DATASET,
print 'TEST DATASET NUM: ',datanum
print 'IOU: ', iu, np.mean(iu)    

        
#     # calc iou
#     hist += fast_hist(label.flatten(),out.flatten(),n_cl)
#     hist = np.array(hist, dtype=np.float32)
#     iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
#     print hist, iu
#     print '>>>> label:', np.bincount(np.array(label, dtype=np.int64).flatten())
#     print '>>>> out:', np.bincount(np.array(out, dtype=np.int64).flatten())
#     plt.subplot(1,2,1)
#     plt.imshow(out)
#     plt.axis('off')
#     plt.subplot(1,2,2)
#     plt.imshow(np.array(im, dtype=np.float32))
#     plt.axis('off')
#     plt.show()
#     break



579386_1489892442_0 [ 0.89603168  0.97153115] 0.933781
douyuTV_786909_1488468426_5 [ 0.88764143  0.96536338] 0.926502
douyuTV_864991_1488861682_0 [ 0.93808222  0.98461843] 0.96135
douyuTV_1185196_1488640592_0 [ 0.79761112  0.90111935] 0.849365
579386_1489894365_0 [ 0.79445207  0.91206521] 0.853259
890551_1489806025_0 [ 0.89691174  0.96334147] 0.930127
douyuTV_527949_1488877658_0 [ 0.98325628  0.99267757] 0.987967
douyuTV_259516_1488461589_0 [ 0.92572296  0.94926882] 0.937496
douyuTV_1551804_1488466294_5 [ 0.84577829  0.93357587] 0.889677
douyuTV_660209_1488641433_0 [ 0.91235441  0.983491  ] 0.947923
douyuTV_786909_1488460444_5 [ 0.6400106   0.90562868] 0.77282
douyuTV_1237818_1488464081_0 [ 0.88974857  0.97972333] 0.934736
douyuTV_1658611_1488461015_0 [ 0.88661373  0.99443042] 0.940522
douyuTV_1642432_1488878289_0 [ 0.98090219  0.98423052] 0.982566
1241123_1489681881_0 [ 0.79822326  0.93653923] 0.867381
douyuTV_604944_1489066625_0 [ 0.94243568  0.97508401] 0.95876
684244_1489818844_0 [

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:72: RuntimeWarning: invalid value encountered in divide


douyuTV_527949_1488877658_15 [ 0.81482905  0.99150795] 0.903168
douyuTV_1185196_1488464442_0 [ 0.83951551  0.9154557 ] 0.877486
douyuTV_660209_1488462460_10 [ 0.97532845  0.991669  ] 0.983499
douyuTV_604944_1489066625_5 [ 0.89046681  0.94971108] 0.920089
douyuTV_1300592_1488461529_0 [ 0.92951906  0.98920274] 0.959361
890551_1489633295_0 [ 0.98272991  0.99593198] 0.989331
douyuTV_684244_1489068246_15 [ 0.90939409  0.9556582 ] 0.932526
douyuTV_733789_1488997443_10 [ 0.8398574   0.97951287] 0.909685
976326_1489867436_0 [ 0.91170347  0.97066343] 0.941183
douyuTV_1121781_1488680377_15 [ 0.86912441  0.6930896 ] 0.781107
douyuTV_976326_1488816186_0 [ 0.93797946  0.92043447] 0.929207
douyuTV_1121781_1488863664_5 [ 0.956756    0.94433284] 0.950544
douyuTV_473002_1488467946_0 [ 0.94454288  0.98434442] 0.964444
976326_1489785498_0 [ 0.96901536  0.98911929] 0.979067
1300592_1489847826_0 [ 0.85670525  0.99445975] 0.925583
douyuTV_976326_1488500470_5 [ 0.94067281  0.97812688] 0.9594
douyuTV_1642432_

In [4]:
##  Eval XiJin's model

import cv2
import numpy as np
from PIL import Image
import sys
sys.path.insert(0,'./caffe/python')
import caffe
import os

import scipy.io as sio    
import matplotlib.pyplot as plt  
plt.rcParams['figure.figsize']=[15,15]
import time
import scipy.misc

INPUT_CHANNEL = 4
MEAN = [114.578, 115.294, 108.353]
DATASET = 'douyu_7100'
n_cl = 2

Mean = np.array((114.578, 115.294, 108.353), dtype=np.float32)
Std = 0.015625

# load net
DEPLOY = '/home/dalong/Workspace/human_matting/fcn_seg/xinjin_model/googlenet_seg_fine_full.prototxt'
CAFFEMODEL = '/home/dalong/Workspace/human_matting/fcn_seg/xinjin_model/googlenet_fine_full_5c_1l_weight_iter_28000.caffemodel'
net = caffe.Net(DEPLOY, CAFFEMODEL, caffe.TEST)
caffe.set_mode_gpu();
caffe.set_device(0);

imglist = open('./data/'+DATASET+'/test5000.txt').readlines()
datanum = len(imglist)

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n,n)

hist = 0

for imgname in imglist:
    imgname = imgname.strip()
    
    # load image, switch to BGR, subtract mean, and make dims C x H x W for Caffe
    img = cv2.imread('./data/'+DATASET+'/Images/'+imgname+'.png')
    if img.shape[0] != 600 or img.shape[1]!=376:
        img = cv2.resize(img,(376,600))
    img = np.float32(img)
    img -= Mean
    img *= Std
    img = img.transpose((2,0,1))
    
    # load label
    label = sio.loadmat('./data/'+DATASET+'/Labels/'+imgname+'.mat')['label']
    label = Image.fromarray(np.uint8(label))
    label = label.resize([376,600],Image.NEAREST)
    label = np.array(label, dtype=np.float32)
    
    if INPUT_CHANNEL==4:        
        # load 4-th channel
        cimg = cv2.imread('./data/'+DATASET+'/pred_xijin/{}.png_pred.png'.format(imgname),cv2.IMREAD_GRAYSCALE)
        if cimg is None:
            print 'none cimg!!!', imgname
            continue
        cimg = cv2.resize(cimg,(376,600))
        cm = np.ndarray((1,600,376),dtype=np.float32)
        cimg = np.float32(cimg)
        cimg = (cimg-128)*Std
        cm[0,:,:] = cimg

    net.blobs['data'].reshape(1,*img.shape)
    net.blobs['data'].data[...] = img
    net.blobs['comask'].reshape(1,*cm.shape)
    net.blobs['comask'].data[...] = cm

    # run net and take argmax for prediction
    net.forward()
    out = net.blobs['score'].data[0].argmax(axis=0)

    this_hist =  fast_hist(label.flatten(),out.flatten(),n_cl)
    this_hist = np.array(this_hist, dtype=np.float32)
    this_iu = np.diag(this_hist) / (this_hist.sum(1) + this_hist.sum(0) - np.diag(this_hist))
    if np.isnan(this_iu[0]):
        this_iu[0] = 1
    if np.isnan(this_iu[1]):
        this_iu[1] = 1
    print imgname, this_iu, np.mean(this_iu) # first is neg iou, second is pos iou 
    hist += this_hist
    
hist = np.array(hist, dtype=np.float32)
iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
print '================================= result ================================='
print 'INPUT_CHANNEL: ',INPUT_CHANNEL
print 'MEAN', MEAN
print 'DEPLOY: ',DEPLOY
print 'CAFFEMODEL: ',CAFFEMODEL
print 'TEST DATASET: ',DATASET,
print 'TEST DATASET NUM: ',datanum
print 'IOU: ', iu, np.mean(iu)    






579386_1489892442_0 [ 0.8981691   0.97024673] 0.934208
douyuTV_786909_1488468426_5 [ 0.89374727  0.96543771] 0.929592
douyuTV_864991_1488861682_0 [ 0.93777478  0.98356283] 0.960669
douyuTV_1185196_1488640592_0 [ 0.89849991  0.94620681] 0.922353
579386_1489894365_0 [ 0.82101995  0.92051971] 0.87077
890551_1489806025_0 [ 0.95520139  0.98342401] 0.969313
douyuTV_527949_1488877658_0 [ 0.98676807  0.99418545] 0.990477
douyuTV_259516_1488461589_0 [ 0.89373165  0.92218393] 0.907958
douyuTV_1551804_1488466294_5 [ 0.86480039  0.93931746] 0.902059
douyuTV_660209_1488641433_0 [ 0.91428864  0.98256826] 0.948428
douyuTV_786909_1488460444_5 [ 0.82341439  0.95077503] 0.887095
douyuTV_1237818_1488464081_0 [ 0.9126879   0.98352325] 0.948106
douyuTV_1658611_1488461015_0 [ 0.93582749  0.99660879] 0.966218
douyuTV_1642432_1488878289_0 [ 0.97794384  0.98145366] 0.979699
1241123_1489681881_0 [ 0.88321358  0.96077245] 0.921993
douyuTV_604944_1489066625_0 [ 0.95051187  0.97763896] 0.964075
684244_1489818844_0

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:82: RuntimeWarning: invalid value encountered in divide


douyuTV_527949_1488877658_15 [ 0.86551195  0.99313605] 0.929324
douyuTV_1185196_1488464442_0 [ 0.90354759  0.94798535] 0.925766
douyuTV_660209_1488462460_10 [ 0.92356563  0.97212625] 0.947846
douyuTV_604944_1489066625_5 [ 0.88908505  0.94149512] 0.91529
douyuTV_1300592_1488461529_0 [ 0.93286198  0.98959422] 0.961228
890551_1489633295_0 [ 0.98059702  0.99537987] 0.987988
douyuTV_684244_1489068246_15 [ 0.95098424  0.9767189 ] 0.963852
douyuTV_733789_1488997443_10 [ 0.89001667  0.98527092] 0.937644
976326_1489867436_0 [ 0.95148754  0.98284358] 0.967166
douyuTV_1121781_1488680377_15 [ 0.91758239  0.72142029] 0.819501
douyuTV_976326_1488816186_0 [ 0.84939492  0.76825613] 0.808825
douyuTV_1121781_1488863664_5 [ 0.80772179  0.685004  ] 0.746363
douyuTV_473002_1488467946_0 [ 0.81559765  0.94105518] 0.878326
976326_1489785498_0 [ 0.95918334  0.9853949 ] 0.972289
1300592_1489847826_0 [ 0.79593176  0.99145299] 0.893692
douyuTV_976326_1488500470_5 [ 0.93749899  0.97651517] 0.957007
douyuTV_1642432